In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import requests
import os
import scipy.signal
import scipy.stats
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.model_selection import KFold
from dateutil.relativedelta import relativedelta
import time
from dateutil.tz import gettz
from datetime import timedelta
import calendar
import datetime
# from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
# import xgboost as xgb
%matplotlib inline
pd.set_option('display.max_rows', 50000)
pd.set_option('display.max_columns', 50000)


chi = [6.635,9.21,11.345,13.277,15.086,16.812,18.475,20.09,21.666,23.209,24.725,26.217,27.688,29.141,30.578,32,33.409,34.805,36.191,37.566,38.932,40.289,41.638,42.98,44.314,45.642,46.963,48.278,49.588,50.892,52.191,53.486,54.776,56.061,57.342,58.619,59.893,61.162,62.428,63.691,64.95,66.206,67.459,68.71,69.957,71.201,72.443,73.683,74.919,76.154,77.386,78.616,79.843,81.069,82.292,83.513,84.733,85.95,87.166,88.379,89.591,90.802,92.01,93.217,94.422,95.626,96.828,98.028,99.228,100.425,101.621,102.816,104.01,105.202,106.393,107.583,108.771,109.958,111.144,112.329,113.512,114.695,115.876,117.057,118.236,119.414,120.591,121.767,122.942,124.116,125.289,126.462,127.633,128.803,129.973,131.141,132.309,133.476,134.642,135.807,136.971,138.134,139.297,140.459,141.62,142.78,143.94,145.099,146.257,147.414,148.571,149.727,150.882,152.037,153.191,154.344,155.496,156.648,157.8,158.95,160.1,161.25,162.398,163.546,164.694,165.841,166.987,168.133,169.278,170.423,171.567,172.711,173.854,174.996,176.138,177.28,178.421,179.561,180.701,181.84,182.979,184.118,185.256,186.393,187.53,188.666,189.802,190.938,192.073,193.208,194.342,195.476,196.609,197.742,198.874,200.006,201.138,202.269,203.4,204.53,205.66,206.79,207.919,209.047,210.176,211.304,212.431,213.558,214.685,215.812,216.938,218.063,219.189,220.314,221.438,222.563,223.687,224.81,225.933,227.056,228.179,229.301,230.423,231.544,232.665,233.786,234.907,236.027,237.147,238.266,239.386,240.505,241.623,242.742,243.86,244.977,246.095,247.212,248.329,249.445
]

step = 10
res_step = 20


In [64]:
## Preprocess data for hairdryer and build corresponding model
hairdryer = pd.read_csv('disagData/emily/hairdryerPhilips_power.csv')
hairdryer.drop('ts',axis=1, inplace=True)
hairdryer['label'] = np.nan
hairdryer['label'].iloc[:510]=0
hairdryer['label'].iloc[510:6788]=1
hairdryer['label'].iloc[6788:]=0
hairdryer = hairdryer.groupby(np.arange(len(hairdryer))//res_step).first()
hairdryer = hairdryer.iloc[:-np.mod(hairdryer.shape[0],step)]


## Preprocess data for tv and build corresponding model
telv = pd.read_csv('disagData/emily/LG_power.csv')
telv.drop('ts',axis=1, inplace=True)
telv['label'] = 1
telv = telv.groupby(np.arange(len(telv))//res_step).first()
telv = telv.iloc[:-np.mod(telv.shape[0],step)]



## Preprocess data for laptop and build corresponding model
laptop = pd.read_csv('disagData/emily/laptop_power.csv')
laptop.drop('ts',axis=1, inplace=True)
laptop['label'] = 1
laptop = laptop.groupby(np.arange(len(laptop))//res_step).first()
laptop = laptop.iloc[:-np.mod(laptop.shape[0],step)]



## Preprocess data for fridge and build corresponding model
fridge = pd.read_csv('disagData/emily/fridge_power.csv')
fridge.drop('ts',axis=1, inplace=True)
fridge['label'] = 1
fridge = fridge.groupby(np.arange(len(fridge))//res_step).first()
fridge = fridge.iloc[:-np.mod(fridge.shape[0],step)]
print(fridge.shape)



(290, 6)


In [65]:
## Preprocess data for hairdryer and build corresponding model

hd = pd.concat([fridge,telv,laptop,hairdryer])
hd['label'].iloc[:fridge.shape[0]+laptop.shape[0]+telv.shape[0]]=0

cols = ['pwrA','apwrA']

hairdryer_pr = pd.DataFrame([])
for col in cols:
    hairdryer_pr[col+'_mean'] = hd[col].groupby(np.arange(len(hd))//step).mean()
    hairdryer_pr[col+'_std'] = hd[col].groupby(np.arange(len(hd))//step).std()
    hairdryer_pr[col+'_min'] = hd[col].groupby(np.arange(len(hd))//step).min()
    hairdryer_pr[col+'_max'] = hd[col].groupby(np.arange(len(hd))//step).max()
hairdryer_pr['label'] = hd['label'].groupby(np.arange(len(hd))//step).agg(lambda x:x.value_counts().index[0])

df = hairdryer_pr.copy()

# split dataframe into feature set and labels
y = df['label'].values
X_hd = df.drop('label',axis=1)
# X = preprocessing.MinMaxScaler().fit_transform(X)

# split into training and test sets and train the model

# mdl_hd = KNeighborsClassifier(n_neighbors=5)
mdl_hd = DecisionTreeClassifier(random_state=0,max_depth=50)
X_train, X_test, y_train, y_test = train_test_split(X_hd, y,
                                                    stratify=y, 
                                                    test_size=0.25)

mdl_hd = mdl_hd.fit(X_train,y_train)

y_pred = mdl_hd.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(acc)

1.0


In [66]:
## Preprocess data for tv and build corresponding model


tv = pd.concat([fridge,laptop,hairdryer,telv])
tv['label'].iloc[:fridge.shape[0]+laptop.shape[0]+hairdryer.shape[0]]=0
tv.reset_index(inplace=True, drop=True)

cols = ['pwrA','apwrA']

tv_pr = pd.DataFrame([])
for col in cols:
    tv_pr[col+'_mean'] = tv[col].groupby(np.arange(len(tv))//step).mean()
    tv_pr[col+'_std'] = tv[col].groupby(np.arange(len(tv))//step).std()
    tv_pr[col+'_min'] = tv[col].groupby(np.arange(len(tv))//step).min()
    tv_pr[col+'_max'] = tv[col].groupby(np.arange(len(tv))//step).max()
tv_pr['label'] = tv['label'].groupby(np.arange(len(tv))//step).agg(lambda x:x.value_counts().index[0])

df = tv_pr.copy()

# split dataframe into feature set and labels
y = df['label'].values
X_tv = df.drop('label',axis=1)
# X = preprocessing.MinMaxScaler().fit_transform(X)

# split into training and test sets and train the model

# mdl_tv = KNeighborsClassifier(n_neighbors=5)
mdl_tv = DecisionTreeClassifier(random_state=0,max_depth=50)

X_train, X_test, y_train, y_test = train_test_split(X_tv, y,
                                                    stratify=y, 
                                                    test_size=0.25)

mdl_tv = mdl_tv.fit(X_train,y_train)

y_pred = mdl_tv.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(acc)

1.0


In [67]:
## Preprocess data for tv and build corresponding model

lp = pd.concat([fridge,telv,hairdryer,laptop])
lp['label'].iloc[:fridge.shape[0]+hairdryer.shape[0]+telv.shape[0]]=0
lp.reset_index(inplace=True, drop=True)

cols = ['pwrA','apwrA']

laptop_pr = pd.DataFrame([])
for col in cols:
    laptop_pr[col+'_mean'] = lp[col].groupby(np.arange(len(lp))//step).mean()
    laptop_pr[col+'_std'] = lp[col].groupby(np.arange(len(lp))//step).std()
    laptop_pr[col+'_min'] = lp[col].groupby(np.arange(len(lp))//step).min()
    laptop_pr[col+'_max'] = lp[col].groupby(np.arange(len(lp))//step).max()
laptop_pr['label'] = lp['label'].groupby(np.arange(len(lp))//step).agg(lambda x:x.value_counts().index[0])

df = laptop_pr.copy()

# split dataframe into feature set and labels
y = df['label'].values
X_lp = df.drop('label',axis=1)
# X = preprocessing.MinMaxScaler().fit_transform(X)

# split into training and test sets and train the model

# mdl_tv = KNeighborsClassifier(n_neighbors=5)
mdl_lp = DecisionTreeClassifier(random_state=0,max_depth=50)

X_train, X_test, y_train, y_test = train_test_split(X_lp, y,
                                                    stratify=y, 
                                                    test_size=0.25)

mdl_lp = mdl_lp.fit(X_train,y_train)

y_pred = mdl_lp.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(acc)

1.0


In [69]:
## Preprocess data for tv and build corresponding model

fr = pd.concat([telv,hairdryer,laptop,fridge])
fr['label'].iloc[:laptop.shape[0]+hairdryer.shape[0]+telv.shape[0]]=0
fr.reset_index(inplace=True, drop=True)

cols = ['pwrA','apwrA']

fridge_pr = pd.DataFrame([])
for col in cols:
    fridge_pr[col+'_mean'] = fr[col].groupby(np.arange(len(fr))//step).mean()
    fridge_pr[col+'_std'] = fr[col].groupby(np.arange(len(fr))//step).std()
    fridge_pr[col+'_min'] = fr[col].groupby(np.arange(len(fr))//step).min()
    fridge_pr[col+'_max'] = fr[col].groupby(np.arange(len(fr))//step).max()
fridge_pr['label'] = fr['label'].groupby(np.arange(len(fr))//step).agg(lambda x:x.value_counts().index[0])

df = fridge_pr.copy()

# split dataframe into feature set and labels
y = df['label'].values
X_fr = df.drop('label',axis=1)
# X = preprocessing.MinMaxScaler().fit_transform(X)

# split into training and test sets and train the model

# mdl_tv = KNeighborsClassifier(n_neighbors=5)
mdl_fr = DecisionTreeClassifier(random_state=0,max_depth=50)

X_train, X_test, y_train, y_test = train_test_split(X_fr, y,
                                                    stratify=y, 
                                                    test_size=0.25)

mdl_fr = mdl_fr.fit(X_train,y_train)

y_pred = mdl_fr.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(acc)

0.967741935483871


In [92]:
dict_classifiers = {
    "Hairdryer": mdl_hd,
    "laptop": mdl_lp,
    "TV": mdl_tv, 
    "Fridge": mdl_fr,
    }

In [123]:
# Read tv and hairdryer data
df = pd.read_csv('disagData/emily/tv_hairdryer_power.csv')
df.drop('ts',axis=1, inplace=True)

df = df.groupby(np.arange(len(df))//res_step).first()


df['totalpwr'] =df['pwrA']+df['rpwrA']* 1j 


# throw out the last elements, in order to have a round division 

mod = np.mod(df.shape[0],step)

if mod>0:
    df = df.iloc[:-mod]
active = df['pwrA'].as_matrix()
active= np.reshape(active,(-1,step))

totalpwr = df['totalpwr'].as_matrix()
totalpwr = np.reshape(totalpwr,(-1,step))

events=np.array([])
events = np.append(events,0)

for i in range(0,totalpwr.shape[0]-1):
    lgof = np.absolute(np.sum((totalpwr[i+1]-totalpwr[i])**2/totalpwr[i]))
    if lgof> chi[step-2]:
        diff = np.absolute(np.mean(active[i+1])-np.mean(active[i-5:i]))
        if diff>15:
            lgof=1
        else:
            lgof=0
    else:
        lgof= 0
    events = np.append(events,lgof)

cols = ['pwrA','apwrA']

df_pr = pd.DataFrame([])
for col in cols:
    df_pr[col+'_mean'] = df[col].groupby(np.arange(len(df))//step).mean()
    df_pr[col+'_std'] = df[col].groupby(np.arange(len(df))//step).std()
    df_pr[col+'_min'] = df[col].groupby(np.arange(len(df))//step).min()
    df_pr[col+'_max'] = df[col].groupby(np.arange(len(df))//step).max()
    
df_pr['events']=events

df_pr

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/home/emilyp/.local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/emilyp/.local/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,pwrA_mean,pwrA_std,pwrA_min,pwrA_max,apwrA_mean,apwrA_std,apwrA_min,apwrA_max,events
0,20.11432,4.237401,12.7139,23.9912,36.41776,5.931699,23.3142,41.8897,0.0
1,21.92728,0.992992,20.0450,22.7528,38.87797,1.562477,35.8960,40.1560,0.0
2,20.00203,0.360344,19.5991,20.6394,35.79365,0.644708,35.1365,36.9858,0.0
3,20.54526,0.581422,19.6322,21.2833,36.63570,0.832698,35.5327,37.4976,0.0
4,20.19688,0.714417,19.6157,21.4980,36.10900,1.092886,35.1695,37.8774,0.0
5,20.60305,1.109293,19.3845,21.8117,36.57792,1.697037,34.7072,38.4553,0.0
6,21.88930,0.107857,21.7622,22.0759,38.58740,0.245651,38.3067,39.0332,0.0
7,21.09508,0.996032,19.7312,22.1749,37.41507,1.733731,35.2686,39.3634,0.0
8,19.97890,0.365321,19.5001,20.6889,35.53277,0.512417,34.8393,36.5730,0.0
9,19.70648,0.094126,19.5166,19.8468,35.10014,0.068657,35.0374,35.2025,0.0


In [124]:
fev = 0
appliances = {}

for i in range(1,df_pr.shape[0]):
    df2 = df_pr.copy()

    if (df_pr['events'].iloc[i]==1.0 and df_pr['events'].iloc[i-1] == 0.0):
        fev = 1
        new_ev=1 # new event detected
        print('New event')
        
        prev_pwrA_mean = df2['pwrA_mean'].iloc[i-1]# - prev_pwrA_mean)
        prev_pwrA_std = df2['pwrA_std'].iloc[i-1] #- prev_pwrA_std)
        prev_pwrA_min = df2['pwrA_min'].iloc[i-1] #- prev_pwrA_min)
        prev_pwrA_max = df2['pwrA_max'].iloc[i-1] #- prev_pwrA_max)
        prev_apwrA_mean = df2['apwrA_mean'].iloc[i-1] #- prev_apwrA_mean)
        prev_apwrA_std = df2['apwrA_std'].iloc[i-1] #- prev_apwrA_std)
        prev_apwrA_min = df2['apwrA_min'].iloc[i-1] #- prev_apwrA_min)
        prev_apwrA_max = df2['apwrA_max'].iloc[i-1] #- prev_apwrA_max)
    else:
        new_ev=0
        
        
        
    if fev>0:
        X[0][0] = np.abs(df2['pwrA_mean'].iloc[i] - prev_pwrA_mean)
        X[0][1] = np.abs(df2['pwrA_std'].iloc[i] - prev_pwrA_std)
        X[0][2] = np.abs(df2['pwrA_min'].iloc[i] - prev_pwrA_min)
        X[0][3] = np.abs(df2['pwrA_max'].iloc[i] - prev_pwrA_max)
        X[0][4] = np.abs(df2['apwrA_mean'].iloc[i] - prev_apwrA_mean)
        X[0][5] = np.abs(df2['apwrA_std'].iloc[i] - prev_apwrA_std)
        X[0][6] = np.abs(df2['apwrA_min'].iloc[i] - prev_apwrA_min)
        X[0][7] = np.abs(df2['apwrA_max'].iloc[i] - prev_apwrA_max)
    else:
        X[0][0] = df2['pwrA_mean'].iloc[i]
        X[0][1] = df2['pwrA_std'].iloc[i] 
        X[0][2] = df2['pwrA_min'].iloc[i] 
        X[0][3] = df2['pwrA_max'].iloc[i]
        X[0][4] = df2['apwrA_mean'].iloc[i] 
        X[0][5] = df2['apwrA_std'].iloc[i] 
        X[0][6] = df2['apwrA_min'].iloc[i] 
        X[0][7] = df2['apwrA_max'].iloc[i] 
     
            
#     X = df2.iloc[i][:-1].values.reshape(1,-1)
#     print(X)
    for ap, mdl in dict_classifiers.items():
#         print('Answer of classifier is:',mdl.predict(X))
        if mdl.predict(X)>0:
            if new_ev==1:
                if (df2['pwrA_mean'].iloc[i] - prev_pwrA_mean)>0:
                    appliances[ap] = X[0][0]
                else:
                    del appliances[ap]
            else:
                
                appliances[ap] = X[0][0]
#                 if ap not in appliances:
                  
                
        
    print(appliances)

{'TV': 21.927279999999996}
{'TV': 20.002029999999998}
{'TV': 20.54526}
{'TV': 20.19688}
{'TV': 20.60305}
{'TV': 21.8893}
{'TV': 21.09508}
{'TV': 19.9789}
{'TV': 19.70648}
New event
{'TV': 19.70648, 'Hairdryer': 1541.9164999999998}
{'TV': 19.70648, 'Hairdryer': 1921.7602599999998}
{'TV': 19.70648, 'Hairdryer': 1914.0691899999997}
{'TV': 19.70648, 'Hairdryer': 1910.93698}


In [106]:
# Read fridge and tv data
df = pd.read_csv('disagData/emily/fridge_tv_power.csv')
df.drop('ts',axis=1, inplace=True)
df = df.groupby(np.arange(len(df))//res_step).first()


df['totalpwr'] =df['pwrA']+df['rpwrA']* 1j 


# throw out the last elements, in order to have a round division 
mod = np.mod(df.shape[0],step)

if mod>0:
    df = df.iloc[:-mod]

active = df['pwrA'].as_matrix()
active= np.reshape(active,(-1,step))

totalpwr = df['totalpwr'].as_matrix()
totalpwr = np.reshape(totalpwr,(-1,step))

events=np.array([])
events = np.append(events,0)

for i in range(0,totalpwr.shape[0]-1):
    lgof = np.absolute(np.sum((totalpwr[i+1]-totalpwr[i])**2/totalpwr[i]))
    if lgof> chi[step-2]:
        diff = np.absolute(np.mean(active[i+1])-np.mean(active[i-5:i]))
        if diff>15:
            lgof=1
        else:
            lgof=0
    else:
        lgof= 0
    events = np.append(events,lgof)

cols = ['pwrA','apwrA']

df_pr = pd.DataFrame([])
for col in cols:
    df_pr[col+'_mean'] = df[col].groupby(np.arange(len(df))//step).mean()
    df_pr[col+'_std'] = df[col].groupby(np.arange(len(df))//step).std()
    df_pr[col+'_min'] = df[col].groupby(np.arange(len(df))//step).min()
    df_pr[col+'_max'] = df[col].groupby(np.arange(len(df))//step).max()
    
df_pr['events']=events

df_pr

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/home/emilyp/.local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/emilyp/.local/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,pwrA_mean,pwrA_std,pwrA_min,pwrA_max,apwrA_mean,apwrA_std,apwrA_min,apwrA_max,events
0,68.35101,6.643864,58.8965,77.0757,82.20416,6.825106,73.2120,91.0609,0.0
1,57.12320,0.334885,56.3703,57.5756,70.38027,0.242945,70.0088,70.7353,0.0
2,59.40178,0.677378,57.8728,60.0028,72.21800,0.530366,71.2306,72.8818,0.0
3,59.45628,0.431719,58.6819,59.8377,72.24938,0.383221,71.5939,72.6341,0.0
4,60.01766,0.342082,59.2763,60.5477,72.70178,0.373066,71.7590,73.1294,0.0
5,66.13847,8.538238,60.0193,83.2675,78.37680,8.188415,72.8983,96.3281,0.0
6,80.22443,4.462948,72.3699,84.0105,93.52279,4.460152,84.1756,97.1041,1.0
7,82.26692,0.204343,81.9961,82.6401,95.47115,0.350659,94.8090,96.0144,0.0
8,81.66751,0.340308,81.1375,82.0952,94.96753,0.340359,94.4623,95.5025,0.0
9,81.22336,0.169251,81.0219,81.5668,94.45403,0.150583,94.2476,94.7760,0.0


In [121]:
fev = 0
appliances = {}

for i in range(1,df_pr.shape[0]):
    df2 = df_pr.copy()

    if (df_pr['events'].iloc[i]==1.0 and df_pr['events'].iloc[i-1] == 0.0):
        fev = 1
        new_ev=1 # new event detected
        print('New event')
        
        prev_pwrA_mean = df2['pwrA_mean'].iloc[i-1]# - prev_pwrA_mean)
        prev_pwrA_std = df2['pwrA_std'].iloc[i-1] #- prev_pwrA_std)
        prev_pwrA_min = df2['pwrA_min'].iloc[i-1] #- prev_pwrA_min)
        prev_pwrA_max = df2['pwrA_max'].iloc[i-1] #- prev_pwrA_max)
        prev_apwrA_mean = df2['apwrA_mean'].iloc[i-1] #- prev_apwrA_mean)
        prev_apwrA_std = df2['apwrA_std'].iloc[i-1] #- prev_apwrA_std)
        prev_apwrA_min = df2['apwrA_min'].iloc[i-1] #- prev_apwrA_min)
        prev_apwrA_max = df2['apwrA_max'].iloc[i-1] #- prev_apwrA_max)
    else:
        new_ev=0
        
        
        
    if fev>0:
        X[0][0] = np.abs(df2['pwrA_mean'].iloc[i] - prev_pwrA_mean)
        X[0][1] = np.abs(df2['pwrA_std'].iloc[i] - prev_pwrA_std)
        X[0][2] = np.abs(df2['pwrA_min'].iloc[i] - prev_pwrA_min)
        X[0][3] = np.abs(df2['pwrA_max'].iloc[i] - prev_pwrA_max)
        X[0][4] = np.abs(df2['apwrA_mean'].iloc[i] - prev_apwrA_mean)
        X[0][5] = np.abs(df2['apwrA_std'].iloc[i] - prev_apwrA_std)
        X[0][6] = np.abs(df2['apwrA_min'].iloc[i] - prev_apwrA_min)
        X[0][7] = np.abs(df2['apwrA_max'].iloc[i] - prev_apwrA_max)
    else:
        X[0][0] = df2['pwrA_mean'].iloc[i]
        X[0][1] = df2['pwrA_std'].iloc[i] 
        X[0][2] = df2['pwrA_min'].iloc[i] 
        X[0][3] = df2['pwrA_max'].iloc[i]
        X[0][4] = df2['apwrA_mean'].iloc[i] 
        X[0][5] = df2['apwrA_std'].iloc[i] 
        X[0][6] = df2['apwrA_min'].iloc[i] 
        X[0][7] = df2['apwrA_max'].iloc[i] 
     
            
#     X = df2.iloc[i][:-1].values.reshape(1,-1)
#     print(X)
    for ap, mdl in dict_classifiers.items():
#         print('Answer of classifier is:',mdl.predict(X))
        if mdl.predict(X)>0:
            if new_ev==1:
                if (df2['pwrA_mean'].iloc[i] - prev_pwrA_mean)>0:
                    appliances[ap] = X[0][0]
                else:
                    del appliances[ap]
            else:
                
                appliances[ap] = X[0][0]
#                 if ap not in appliances:
                  
                
        
    print(appliances)

{'Fridge': 57.1232}
{'Fridge': 57.1232}
{'Fridge': 59.45628000000001}
{'Fridge': 60.01766}
{'Fridge': 60.01766}
New event
{'Fridge': 60.01766}
{'Fridge': 60.01766, 'TV': 16.128449999999987}
{'Fridge': 60.01766, 'TV': 15.529039999999995}
{'Fridge': 60.01766, 'TV': 15.084889999999987}
{'Fridge': 60.01766, 'TV': 15.212029999999984}
{'Fridge': 60.01766, 'TV': 15.159199999999998}
